In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv

#選択行削除
import re
from xml.sax.saxutils import unescape

In [3]:
def normalize_XBRL(XBRL):
    with open(XBRL, encoding='utf-8') as f:
        html = f.read()
    # 「（」「）」はすべて半角に
    html = html.replace("（", "(")
    html = html.replace("）", ")")
    #tableタグだけ残す，それ以外のタグで囲まれていない部分を削除
    html  = re.sub("<table", "table\n<", html)
    html  = re.sub("</table", "tableend\n<", html)
    html  = re.sub("<br />", "", html)
    html  = re.sub("<td", "td<", html)
    html  = re.sub("</td>", "tdend\n", html)
    html  = re.sub("</tr>", "trend", html)
    
    text=""
    for i in html:
        text=text+i
        if i ==">":
            text=re.sub("<.*>", "", text)
            text=re.sub("{.*}", "", text)
            if text[-1:]!="\n":
                text = text+"\n"
    text1=text

    #テキスト処理
    text1=re.sub("&#160;", "None", text1)
    text1=re.sub("△\n", "-", text1)
    text1=re.sub("△", "-", text1)
    text1=re.sub(".root", "-", text1)
    text1=re.sub(",", "", text1)
    text1=re.sub("．", ".", text1)
    text1=re.sub("\ufeff\n", "", text1)
    text1=re.sub("－", "None", text1)
    text1=re.sub("：", "", text1)
    text1=re.sub(" ", "", text1)
    text1=re.sub("\n\n", "\n", text1)
    text1=re.sub("　", "", text1)
    text1=re.sub("０", "0", text1)
    text1=re.sub("１", "1", text1)
    text1=re.sub("２", "2", text1)
    text1=re.sub("３", "3", text1)
    text1=re.sub("４", "4", text1)
    text1=re.sub("５", "5", text1)
    text1=re.sub("６", "6", text1)
    text1=re.sub("７", "7", text1)
    text1=re.sub("８", "8", text1)
    text1=re.sub("９", "9", text1)
    

    return(text1)

In [4]:
def text_tag(text):
    #予測がいつまでなのかを取得(基本的に　３．　OO年度連結業績予測などになっているため，それを元に取得)
    pre_cnt=0
    pre = ""
    for i in range(len(text),-1,-1):
        if "連結業績予想(" in text[i-7:i]:
            pre_cnt=i
            break
    for i in range(pre_cnt,-1,-1):
        if "." in text[i-1:i]:
            pre = text[i:pre_cnt-1]
            break

    #tdで囲まれた部分の改行は削除して，tdを削除
    tx=""
    text1=""
    td_flag=False
    for i in text:
        #print(i)
        if "tdend" in text1:
            td_flag=False
        if "td" in text1:
            td_flag=True

        if td_flag:
            if i=="\n":
                i=""
        text1+=i
    text1=re.sub("tdend", "", text1)
    text1=re.sub("td", "\n", text1)
    text1=re.sub(r"\n+", "\n", text1)
    return(text1,pre)

In [5]:
def create_text_list(text):

    #tableタグで囲まれた部分をリストに入れる
    text_list = []
    table_cnt_first=0
    table_cnt_end=0
    non_text_list = []
    non_table_cnt_first=0
    append_text=""
    non_text=""
    for i in range(1,len(text)):
        if text[i-8:i] == "tableend":
        #tableタグに囲まれた文章をlistに追加
            table_cnt_end = i-8
            append_text=text[table_cnt_first:table_cnt_end]+"\n"
            append_text=re.sub("trend\ntrend","trend\n",append_text)
            append_text=re.sub("tableendtable","",append_text)
            append_text=re.sub(r'\n+', "\n",append_text)
            append_text=re.sub(r'\ntrend\nNone',"",append_text)
            #append_text=re.sub(r'\nNonetrend',"",append_text)
            text_list.append(append_text)
            table_cnt_first=0
            table_cnt_end=0
            #tableタグに囲まれている部分を追加したので囲まれていない部分のカウント初期化
            non_table_cnt_first=i

    #tableタグの始まりの判定    
        elif text[i-5:i] == "table":
            if table_cnt_first==0:
                table_cnt_first=i

            #tableタグに囲まれていない部分をlistに追加
            non_text=text[non_table_cnt_first:i-5]
            non_text_list.append(non_text)
    
    #listの中に（注）がある部分は削除する         
    pop_list=[]
    cnt=0
    for i in text_list:

        #if "(注)" in i :
          #  pop_list.append(cnt)
        if "(参考)" in i:
            pop_list.append(cnt)
        elif "(役職名)" in i:
            pop_list.append(cnt)
        cnt +=1

    for i in pop_list[::-1]:
        #print(i)
        text_list.pop(i)
    return(text_list,non_text_list)

In [6]:
def create_table_list(text_list,column_dic):
    new_li=[]
    n_cnt=0
    col=""
    column_li=[i for i in column_dic.keys()]
    #一番可能性が高い表をlistのトップに持ってくる
    for cnt,table in enumerate(text_list):
        n_cnt=0
        col=""
        for i,word in enumerate(table,1):
            if word=="\n":
                #columnになる可能性のある単語がcolumn辞書に存在する場合は高い確率で目的の表であると考えられる
                col = table[n_cnt:i-1]
                #col= re.sub("trend","",col)
                if col in column_li:
                    new_li.append(table)
                    break
                n_cnt=i
    return(new_li)

In [7]:
def create_list(table_list):
    number_list = []
    col_list=[]
    number = []
    col=""
    num_cnt=0
    col_cnt=0
    len_col=0
    add_col=0
    non_cnt=0
    for table in table_list:
        if "table" in table:
            table=table[table.find("table")+5:]
        for i in range(0,len(table)):
            if table[i:i+5] !="trend":
                if table[i:i+1] == "\n":
                    col=table[col_cnt+1:i]
                    add_col+=1
                    if col=="" and col_list!=[] and len_col==0:
                        len_col=len(col_list)
                    if col!="":
                        col_list.append(col)
                    col_cnt=i
                
            else:
                col=table[col_cnt+1:i]
                col_list.append(col)
                #追加したコラムが１つだけなら削除してやり直し
                if add_col==1:
                    add_col=0
                    col_list.pop(-1)
                    col_cnt=i+5
                    continue
                add_col=0
                num_cnt=i+5
                break
                
        #col_listの先頭がNoneでなければNoneを追加
        if (col_list!=[]) and col_list[non_cnt]!="None":
            #col_list.insert(len(column),"None")
            col_list.insert(non_cnt,"None")
            non_cnt=len(col_list)
            
        #column以降にある数字の部分を回す
        for i in range(num_cnt,len(table)+1):
            if table[i:i+1] == "\n":
                if table[i-5:i]=="trend":
                    if number!=[]:
                        number.append(table[num_cnt+1:i-5])
                        number_list.append(number)
                        number=[]
                        num_cnt=i

                elif table[i-5:i]!="trend":
                    number.append(table[num_cnt+1:i])
                    num_cnt=i
                    
    #[col_list.insert(k, col_list[k]) for k in range(len_col-1,-1,-1)]
    
    return(col_list,number_list)

In [8]:
def create_NoneAndCol(column,number):
    #columnをNoneを起点に分ける
    number_list=[]
    column_list=[]
    pop_list=[]
    cn=1
    for i in range(1,len(column)):
        if column[i] =="None":
            column_list.append(column[cn:i])
            cn=i+1
        elif i==len(column)-1:
            column_list.append(column[cn:i+1])
            
            
    #numberにNoneを入れる
    for cnt,num in enumerate(number):
        #if len(num) <3:
          #  pop_list.append(cnt)
        if ("円銭" in num) or ("百万円" in num):
            if "None" not in num:
                num.insert(0,"None")
        
        #後々Noneがあるとエラーになるのでいらないものは削除
        if num[0] != "None":
            num[0] = re.sub("None","",num[0])
        number_list.append(num)
        

    for i in pop_list[::-1]:
        #print(i)
        number_list.pop(i)
    return(column_list,number_list)

In [9]:
def create_index(number):
    index_list=[[]*i for i in range(0,len(number))]
    in_cnt=0
    index_list[in_cnt].append("None")

    for i in range(1,len(number)):
        if number[i][0]== "None":
            in_cnt+=1
            index_list[in_cnt].append("None")
        else:
            index_list[in_cnt].append(number[i][0])
    return(index_list)

In [10]:
def create_new_number_list(number,column_list):
    new_num=[]
    cn=0
    for cnt,num in enumerate(number):
        if num[0]== "None":
            if len(new_num)!=len(column_list):
                if len(column_list[cn])>2:

                    for i in range(0,len(num),2):
                        if i>len(column_list[cn])-1:
                            continue
                        if (len(num)-1) > (len(column_list[cn])):
                            column_list[cn].insert(i,column_list[cn][i])

                        else:
                            break
                new_num.append([column_list[cn][k]+num[k+1] for k in range(0,len(column_list[cn]))])
                new_num[cn].insert(0,"None")
                number[cnt]=new_num[cn]
                if cn!=len(column_list)-1:
                    cn+=1
    
    return(number)

In [11]:
def create_DataFrame(number,index_list):
    #最大でも４つまでの表しか作成しない．目的の表は２つだけなので
    new_df1 = pd.DataFrame
    new_df2 = pd.DataFrame
    new_df3 = pd.DataFrame
    new_df4=pd.DataFrame
    new_df = pd.DataFrame
    cn=1
    num_cnt=0
    for cnt,index in enumerate(index_list):

        if index!=[] and cn==1:
            new_df1=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            cn+=1
            continue
        elif index!=[] and cn==2:
            new_df2=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            #インデックスが同じなら合わせる
            if index==index_list[cnt-1]:
                new_df=pd.concat([new_df1,new_df2],axis=1)
            else:
                cn+=1

        elif index!=[] and cn==3:
            new_df3=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            num_cnt+=len(index)
            cn+=1
        elif index!=[] and cn==4:
            new_df4=pd.DataFrame([number[k][1:] for k in range(num_cnt,len(index)+num_cnt)],index=index)
            break
    #dfがNoneだったらdf1を出力
    if str(type(new_df))=="<class 'type'>": 
        return(new_df1,new_df2,new_df3,new_df4)
          
    else:
        return(new_df,new_df2,new_df3,new_df4)

In [12]:
def reset_column(df1,df2):
    sh1=df1.shape
    sh2 = df2.shape
    #column1をリセット
    df1.columns = range(sh1[1])
    #column2をリセット
    df2.columns = range(sh2[1])
    
    return(df1,df2)

In [13]:
def reset_column1(df1):
    sh1=df1.shape

    #column1をリセット
    df1.columns = range(sh1[1])

    return(df1)

In [202]:
def select_df(df2,df3,df4,pre_term):
    
    try:
        if ("通期" in str(df2.index)) or( "予想" in str(df2.index ))or("累計" in str(df2.index))or("四半期" in str(df2.index)):
            return(df2)
        elif ("通期" in str(df3.index)) or( "予想" in str(df3.index ))or("累計" in str(df3.index)): 
            return(df3)
        elif ("通期" in str(df4.index)) or( "予想" in str(df4.index ))or("累計" in str(df4.index)):
            return(df4)
        
        else:
            index_list=[]
            index_list = [i for i in df2.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df2)
                pass
            
            index_list = [i for i in df3.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df3)
                pass
            
            index_list = [i for i in df4.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df4)
                pass
    except:
        pass

In [15]:
def select_df2(df,df2,df3,df4):
    if str(type(df))=="<class 'NoneType'>":
        try:
            if "累計" in str(df2.index) or "累計" in str(df2.index):
                return(df2)
            elif "累計" in str(df3.index) or "累計" in str(df3.index):
                return(df3)
            elif "累計" in str(df4.index) or "累計" in str(df4.index):
                return(df4)
        except:
            pass

In [16]:
def create_df1_column(df1,column_list):
    df1_col=[ i for i in df1.columns]
    cnt_0 = df1_col.count(0)
    new_col=[]
    cnt=1
    end=0
    try:
        for zero_cnt in range(0,cnt_0):
            for i in range(cnt,len(df1_col)):
                if df1_col[i]==0:
                    #columnとdfの長さが同じ時（表１）
                    if len(column_list[zero_cnt]) == i:
                        new_col= column_list[0]
                    #columnとdfの長さが同じ時（表２）
                    elif (zero_cnt!=0) and (len(column_list[zero_cnt]) == i-end+1):
                        new_col= column_list[0]
                    else:
                        cnt=i+1
                        break
                #エラーが起きないように最後になったらforを抜ける        
                elif i==len(df1_col)-1:
                    cnt=i+3
                    break
            #抜けた後，column数が同じならそのまま加える
            if (len(column_list[zero_cnt]) == i-end+1):
                for k in column_list[zero_cnt]:
                    new_col.append(k)
            else:
                #0の手前までのcolumnを作成
                cn=0
                for j in range(end,cnt-1):
                    if j %2==0:
                        new_col.append(column_list[zero_cnt][cn])
                    else:
                        new_col.append(column_list[zero_cnt][cn])
                        cn+=1
                end=j+2
        df1.loc["index"]=new_col
    except:
        for k in range(0,3):
            if len(new_col)!=len(df1_col):
                new_col.append("None")
            else:
                break
        df1.loc["index"]=new_col
    return(df1,zero_cnt+1)

In [17]:
def create_df_column(df1,column_list,zero_cnt):
    try:
        df_col=[ i for i in df.columns]
        new_col=[]
        cn=0
        for i in range(zero_cnt,len(column_list)):
            new_col=[]
            cn=0
            for j in range(0,len(df_col)):
                if cn < len(column_list[i]):
                    #print(j)
                    if j %2==0:
                        new_col.append(column_list[i][cn])
                        #print(column_list[i][cn])
                    else:
                        new_col.append(column_list[i][cn])
                        #print(column_list[i][cn])
                        cn+=1

            if  len(df_col)==len(new_col):
                df.loc["index"]=new_col
                break
        return(df1)
    except:
        return(df1)

In [18]:
def check_text(text_list,non_text):
    new_text_list=[]
    new_text=""

    for txt in text_list:
        if txt.count("。")>=1:
            new_text+=txt
    new_text_list.append(new_text)
    
    new_text=""
    for txt in non_text:
        if txt.count("。")>=1:
            new_text+=txt
    new_text_list.append(new_text)
    
    return(new_text_list)

In [19]:
def NoPredict_or(NoPredict_text):
    cnt=0
    cnt=0
    for i in NoPredict_text:
        if ("未定"in i)or ("控え" in i) or ("困難" in i)or("M&A" in i) or("コロナウイルス" in i) or("合理的に算出" in i)or("非連結" in i)or("IFRS" in i)or("精査中" in i):
            cnt+=1
        elif ("速やかに開示" in i) or ("開示する予定"in i)or ("ＩＦＲＳ" in i):
            cnt+=1
    if cnt>0:
        return(True)
    else:
        return(False)

In [280]:
def tuki_reset(number):
    for cnt,li in enumerate(number):
        if "通期" in li:
            if li[0]!="通期":
                number[cnt].remove("通期")
                number[cnt+1].insert(0,"通期")
    return(number)

In [21]:
def trend_reset(number):
    for cnt,li in enumerate(number):
        for cnt2,li2 in enumerate(li):
            if "trend" in li2:
                number[cnt].pop(cnt2)
                break
    return(number)

In [22]:
def col_reset(column):
    for cnt,li in enumerate(column):
        pop_list=[]
        for cnt2,li2 in enumerate(li):
            if "※" in li2:
                pop_list.append(cnt2)
        for j in pop_list[::-1]:
            column[cnt].pop(j)
    return(column)

In [23]:
def main(file): 
    col_dic={
    "売上高":"Sales",
    "営業利益":"Operating",
    "事業利益":"Business",
    "経常利益":"Ordinary",
    "親会社株主に帰属する当期純利益":"ParentNetIncome",
    "親会社株主に帰属する四半期純利益":"QuarterParentNetIncome",
    "1株当たり当期純利益":"OneNetIncome",
    "1株当たり四半期純利益":"QuarterOneNetIncome"

    }
    li=[]
    NoPreFlag=False
    errFlag=False
    try:
        text=normalize_XBRL(file)
        text,pre_term = text_tag(text)
        text_list,non_text=create_text_list(text)
        NoPredict_text=check_text(text_list,non_text)
        table_list=create_table_list(text_list,col_dic)
        column,number=create_list(table_list)
        column_list,number = create_NoneAndCol(column,number)
        number=tuki_reset(number)
        number=trend_reset(number)
        index_list=create_index(number)
        df1,df2,df3,df4=create_DataFrame(number,index_list)
        df=select_df(df2,df3,df4,pre_term)
        df1,zero_cnt,=create_df1_column(df1,column_list)
        df=create_df_column(df,column_list,zero_cnt)
        df1=reset_column1(df1)
        if str(type(df))=="<class 'NoneType'>":
            if NoPredict_or(NoPredict_text):
                    NoPreFlag=True
        else:
            df=reset_column1(df)
    except:
        errFlag=True
        if str(type(df))=="<class 'NoneType'>":
            if NoPredict_or(NoPredict_text):
                NoPreFlag =True
                    
    return(df1,df,pre_term,NoPredict_text,NoPreFlag,errFlag,table_list)

In [24]:
#自作関数読み込み
import pre_process.xbrl_process3 as xb

In [29]:
import pickle
out='new2/err/non_list3.pkl'
with open(out, 'rb') as p:
    out_list=pickle.load(p)

In [28]:
col_dic={
    "売上高":"Sales",
    "営業利益":"Operating",
    "事業利益":"Business",
    "経常利益":"Ordinary",
    "親会社株主に帰属する当期純利益":"ParentNetIncome",
    "親会社株主に帰属する四半期純利益":"QuarterParentNetIncome",
    "1株当たり当期純利益":"OneNetIncome",
    "1株当たり四半期純利益":"QuarterOneNetIncome"
}

## 終了
- 34070(0)
- 59330(1:7)
- 48650(8:11)
- 83030(11:12)
- 48000(13)
- 23890(15:20)
- 37930(21:23)
- 30420(24:25)
- 36880(26:38)
- 67400(39:50)
- 94680(51:57)
- 39030(58:61)
- 31330(62)
- 39940(63)
- 29820(64)
- 73390(65)


## 未完了
- 23890(14)

In [392]:
out_list[14]

'xbrl_data/23890/20190206471708.htm'

In [385]:
l=16
text=normalize_XBRL(out_list[l])
text,pre_term = text_tag(text)
text_list,non_text=create_text_list(text)
NoPredict_text=check_text(text_list,non_text)
table_list=create_table_list(text_list,col_dic)
#kari_list=table_list[-1]
#table_list=table_list[0:2]
#table_list.append(kari_list)
column,number=create_list(table_list)
column_list,number = create_NoneAndCol(column,number)
number=tuki_reset(number)
number=trend_reset(number)
index_list=create_index(number)
df1,df2,df3,df4=create_DataFrame(number,index_list)
df=select_df(df2,df3,df4,pre_term)
df1,zero_cnt,=create_df1_column(df1,column_list)
df=create_df_column(df,column_list,zero_cnt)
df1=reset_column1(df1)
if str(type(df))=="<class 'NoneType'>":
    if NoPredict_or(NoPredict_text):
            NoPreFlag=True
else:
    df=reset_column1(df)

In [386]:
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭,円銭,百万円,％,百万円,％
30年12月期第2四半期,43755,9.4,1089,-24.3,842,-48.7,1179,44.8,52.14,51.86,2660,7.5,2020,20.3
29年12月期第2四半期,39999,13.1,1438,23.7,1642,55.2,814,196.0,34.81,34.62,2475,58.0,1679,106.8
index,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する四半期純利益,親会社株主に帰属する四半期純利益,1株当たり四半期純利益,潜在株式調整後1株当たり四半期純利益,潜在株式調整後1株当たり四半期純利益,EBITDA※1,EBITDA※1,EBIT※2


In [387]:
df

,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
通期,91000,10.2,2100,-5.6,1500,-21.9,1400,38.5,61.90
index,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する当期純利益,親会社株主に帰属する当期純利益,1株当たり当期純利益


In [365]:
column_list

[['売上高', '営業利益', '経常利益', '親会社株主に帰属する四半期純利益'],
 ['1株当たり四半期純利益', '潜在株式調整後1株当たり四半期純利益', 'EBITDA※1', 'EBIT※2'],
 ['売上高', '営業利益', '経常利益', '親会社株主に帰属する四半期純利益', 'EBITDA', 'EBIT'],
 ['売上高', '営業利益', '経常利益', '親会社株主に帰属する当期純利益', '1株当たり当期純利益']]

In [286]:
number

[['None', '百万円', '％', '百万円', '％', '百万円', '％', '百万円', '％', '百万円', '％'],
 ['2021年12月期第2四半期',
  '13678',
  '23.6',
  '698',
  '117.4',
  '624',
  '154.7',
  '489',
  '458.9',
  '346',
  '922.3'],
 ['2020年12月期第2四半期',
  '11069',
  'None',
  '321',
  'None',
  '245',
  'None',
  '87',
  'None',
  '33',
  'None'],
 ['None', '売上高', 'EBITDA', '営業利益', '経常利益'],
 ['None', '百万円', '％', '百万円', '％', '百万円', '％', '百万円', '％'],
 ['2021年1月1日～6月30日',
  '13678',
  '42.7',
  '698',
  '173.6',
  '624',
  '247.8',
  '489',
  'None'],
 ['2020年1月1日～6月30日',
  '9588',
  'None',
  '255',
  'None',
  '179',
  'None',
  '16',
  'None'],
 ['None', '円銭', '円銭'],
 ['2021年12月期第2四半期', '8.34', '8.27'],
 ['2020年12月期第2四半期', '0.87', '0.86'],
 ['None', '百万円', '％', '百万円', '％', '百万円', '％', '百万円', '％', '百万円', '％'],
 ['通期',
  '23000',
  '36.6',
  '1100',
  '70.3',
  '600',
  '40.4',
  '600',
  '38.7',
  '380',
  '43.4']]

In [40]:
import os
def create_dic(dic):
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass

In [41]:
out_text='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
out_txt2='new2/last/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
out_txt3='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])

out_csv1='new2/last/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
out_csv2='new2/last/{}/{}_2.csv'.format(out_list[l][10:15],out_list[l][16:-4])
dic1 = 'new2/last/{}'.format(out_list[l][10:15],out_list[l][16:-4])

#フォルダを作成する
create_dic(dic1)
df1.to_csv(out_csv1)
df.to_csv(out_csv2)
with open(out_text,'w') as out:
    out.write(pre_term)

In [389]:
for l in range(15,21):
    text=normalize_XBRL(out_list[l])
    text,pre_term = text_tag(text)
    text_list,non_text=create_text_list(text)
    NoPredict_text=check_text(text_list,non_text)
    table_list=create_table_list(text_list,col_dic)
    #kari_list=table_list[-1]
    #table_list=table_list[0:2]
    #table_list.append(kari_list)
    column,number=create_list(table_list)
    column_list,number = create_NoneAndCol(column,number)
    number=tuki_reset(number)
    number=trend_reset(number)
    index_list=create_index(number)
    df1,df2,df3,df4=create_DataFrame(number,index_list)
    df=select_df(df2,df3,df4,pre_term)
    df1,zero_cnt,=create_df1_column(df1,column_list)
    df=create_df_column(df,column_list,zero_cnt)
    df1=reset_column1(df1)
    if str(type(df))=="<class 'NoneType'>":
        if NoPredict_or(NoPredict_text):
                NoPreFlag=True
    else:
        df=reset_column1(df)
        
    out_text='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt2='new2/last/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt3='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])

    out_csv1='new2/last/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    out_csv2='new2/last/{}/{}_2.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    dic1 = 'new2/last/{}'.format(out_list[l][10:15],out_list[l][16:-4])

    #フォルダを作成する
    create_dic(dic1)
    df1.to_csv(out_csv1)
    df.to_csv(out_csv2)
    with open(out_text,'w') as out:
        out.write(pre_term)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [55]:
table_list

['\n経常収益\n経常利益\n親会社株主に帰属する四半期純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％trend\n30年3月期第3四半期\n286087\n0.3\n40400\n-0.5\n35609\n-17.9trend\n29年3月期第3四半期\n285210\n2.2\n40609\n-19.8\n43397\n-9.8trend\n',
 '\n1株当たり四半期純利益\n潜在株式調整後1株当たり四半期純利益trend\nNone\n円銭\n円銭trend\n30年3月期第3四半期\n137.57\n137.55trend\n29年3月期第3四半期\n166.14\n166.14trend\n']

In [56]:
out_list[l]

'xbrl_data/83030/20180104445808.htm'

In [135]:
for l in range(62,63):
    text=normalize_XBRL(out_list[l])
    text,pre_term = text_tag(text)
    text_list,non_text=create_text_list(text)
    NoPredict_text=check_text(text_list,non_text)
    table_list=create_table_list(text_list,col_dic)
    column,number=create_list(table_list)
    column_list,number = create_NoneAndCol(column,number)
    number=tuki_reset(number)
    number=trend_reset(number)
    index_list=create_index(number)
    df1,df2,df3,df4=create_DataFrame(number,index_list)
    df=select_df(df2,df3,df4,pre_term)
    df1,zero_cnt,=create_df1_column(df1,column_list)
    df=create_df_column(df,column_list,zero_cnt)
    df1=reset_column1(df1)
    if str(type(df))=="<class 'NoneType'>":
        if NoPredict_or(NoPredict_text):
                NoPreFlag=True
    else:
        df=reset_column1(df)
        
    out_text='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt2='new2/last/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
    out_txt3='new2/last_nopre/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])

    out_csv1='new2/last/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    out_csv2='new2/last/{}/{}_2.csv'.format(out_list[l][10:15],out_list[l][16:-4])
    
    out_pkl='new2/last_nopre/{}/{}.pkl'.format(out_list[l][10:15],out_list[l][16:-4])
    
    dic1 = 'new2/last/{}'.format(out_list[l][10:15],out_list[l][16:-4])
    dic2 = 'new2/last_nopre/{}'.format(out_list[l][10:15],out_list[l][16:-4])
    #フォルダを作成する
    create_dic(dic1)
    create_dic(dic2)
    df1.to_csv(out_csv1)

    with open(out_text,'w') as out:
        out.write(pre_term)
        
    with open(out_pkl, 'wb') as p:
        pickle.dump(NoPredict_text, p)

    with open(out_txt3,'w') as out:
        out.write(pre_term) 

In [353]:
out_text='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])
out_txt2='new2/last/{}/{}_NoPre.txt'.format(out_list[l][10:15],out_list[l][16:-4])
out_txt3='new2/last/{}/{}.txt'.format(out_list[l][10:15],out_list[l][16:-4])

out_csv1='new2/last/{}/{}_1.csv'.format(out_list[l][10:15],out_list[l][16:-4])
out_csv2='new2/last/{}/{}_2.csv'.format(out_list[l][10:15],out_list[l][16:-4])
dic1 = 'new2/last/{}'.format(out_list[l][10:15],out_list[l][16:-4])

#フォルダを作成する
create_dic(dic1)
df1.to_csv(out_csv1)
df.to_csv(out_csv2)
with open(out_text,'w') as out:
    out.write(pre_term)

In [4]:
import os

path = 'new2/xbrl_df'

folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

3225


In [7]:
dic={}

In [8]:
for i in folderfile:
    dic[i]=1

In [15]:
str(int(com_list[0]))

'13760'

In [17]:
str(int(com_list[0])) in dic

True

In [11]:
company = pd.read_csv("TDNet_full_data/company_list.csv",encoding="shift-jis",index_col=0)
#上場企業リスト取得
#証券コードのlistを作成
com_list=[]
for i in range(0,len(company)):
    if company["上場区分"][i]=="上場":
        com_list.append(company["証券コード"][i])
    #if len(com_list)==3882:
        #print(i)
#最初に書いてある，証券コードが記入されていない企業が最後に存在するため削除
com_list = com_list[:-1]

In [20]:
c_list=[]
for com in com_list:
    if not str(int(com)) in dic:
        c_list.append(com)

In [43]:
len(com_list)

3881

In [27]:
len(c_list)

656

In [41]:
import json
#for i in (range(0,10)):
#使用するコードのllistを作成，合致するコードが一つでもあれば取得
pdf_code_list = [[]*i for i in range(0,len(c_list))]
for c in (range(0,len(c_list))):
    #indexを開く
    json_open= open('TDNet_full_data/API_index/{}.json'.format(int(c_list[c])), 'r')
    js = json.load(json_open)
   # print(js)
    #break
    
    for i in range(0,len(js["publiclyList"])):
        for j in range(0,len(js["publiclyList"][i]["disclosureItems"])):
            if (js["publiclyList"][i]["disclosureItems"][j][2:] == "301" or 
                js["publiclyList"][i]["disclosureItems"][j][2:] == "304" or 
                js["publiclyList"][i]["disclosureItems"][j][2:] == "307" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "310" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "313" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "350" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "352" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "354" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "356" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "358" or
                js["publiclyList"][i]["disclosureItems"][j][2:] == "360"):
                    if (js["publiclyList"][j]["xbrlFlag"]=="1") and (js["publiclyList"][j]["pdfSummaryFlag"]=="1"):
                        pdf_code_list[c].append(js["publiclyList"][j]["disclosureNumber"])
                        break
    print(js["publiclyList"][j])

{'code': '13750', 'name': '雪国まいたけ', 'disclosedDate': '2022-06-03', 'disclosedTime': '08:00:00', 'handlingType': None, 'disclosureNumber': '20220602569156', 'modifiedHistory': '1', 'title': 'Notice of Convocation Annual General Meeting 2022', 'disclosureItems': ['11737'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '19710', 'name': '中央ビルト', 'disclosedDate': '2022-05-20', 'disclosedTime': '15:00:00', 'handlingType': None, 'disclosureNumber': '20220518551078', 'modifiedHistory': '1', 'title': '定款一部変更に関するお知らせ', 'disclosureItems': ['11140'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '19720', 'name': '三晃金', 'disclosedDate': '2022-05-31', 'disclosedTime': '08:00:00', 'handlingType': None, 'disclosureNumber': '20220530562666', 'modifiedHistory': '1', 'title': '2022年定時株主総会招集通知', 'disclosureItems': ['11737'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '18470', 'name': 'イチケン', 'disclosedDate': '2022-06-03', 'di

{'code': '74620', 'name': 'ＣＡＰＩＴＡ', 'disclosedDate': '2022-05-31', 'disclosedTime': '17:40:00', 'handlingType': None, 'disclosureNumber': '20220531565540', 'modifiedHistory': '1', 'title': '販売用不動産の取得に関するお知らせ', 'disclosureItems': ['11199'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '75050', 'name': '扶桑電通', 'disclosedDate': '2022-05-13', 'disclosedTime': '14:31:00', 'handlingType': None, 'disclosureNumber': '20220421525042', 'modifiedHistory': '1', 'title': 'コーポレート・ガバナンスに関する報告書 2022/05/13', 'disclosureItems': ['11744'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '1'}
{'code': '75060', 'name': 'ハウスローゼ', 'disclosedDate': '2022-05-31', 'disclosedTime': '08:00:00', 'handlingType': None, 'disclosureNumber': '20220527561761', 'modifiedHistory': '1', 'title': '2022年定時株主総会招集通知', 'disclosureItems': ['11737'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '75120', 'name': 'イオン北海', 'disclosedDate': '2022-05-31', 'disclosedTime':

{'code': '14440', 'name': 'ニッソウ', 'disclosedDate': '2022-04-07', 'disclosedTime': '15:30:00', 'handlingType': None, 'disclosureNumber': '20220404516789', 'modifiedHistory': '1', 'title': 'アジア太平洋地域の急成長企業ランキング選出に関するお知らせ', 'disclosureItems': ['11802'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '44320', 'name': 'ウイングアーク', 'disclosedDate': '2022-05-26', 'disclosedTime': '10:39:00', 'handlingType': None, 'disclosureNumber': '20220525558075', 'modifiedHistory': '1', 'title': '定款 2022/05/25', 'disclosureItems': ['11729'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '43840', 'name': 'ラクスル', 'disclosedDate': '2022-04-13', 'disclosedTime': '08:00:00', 'handlingType': None, 'disclosureNumber': '20220412520791', 'modifiedHistory': '1', 'title': '「FTSE Blossom Japan Sector Relative Index」の構成銘柄に選定', 'disclosureItems': ['11802'], 'pdfGeneralFlag': '1', 'pdfSummaryFlag': '0', 'xbrlFlag': '0'}
{'code': '65800', 'name': 'Ｇ－ライトアップ', 'disclosedDate'

In [42]:
pdf_code_list

[[],
 ['20220520552778', '20220520552778'],
 ['20220427529022', '20220427529022'],
 [],
 ['20220427530329'],
 ['20220510536432', '20220510536432', '20220510536432', '20220510536432'],
 [],
 [],
 [],
 ['20220427529740'],
 [],
 [],
 ['20220209584489', '20220209584489'],
 [],
 ['20220510536855', '20220510536855', '20220510536855'],
 [],
 [],
 [],
 [],
 ['20220510538075'],
 [],
 [],
 ['20220512544827', '20220512544827', '20220512544827'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['20220512542093', '20220512542093', '20220512542093'],
 [],
 [],
 [],
 ['20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277',
  '20220418523277'],
 [],
 [],
 ['20220427529611'],
 [],
 [],
 [],
 ['20220106564707'],
 ['20220510537933'],
 [],
 ['20211221557336', '20211221557336', '20211221557336', '20211221557336'],
 ['20220511540375'],
 [],
 [],
 ['20220307501253'],
 [],
 ['20220511541087'],
 [],
 ['2022

In [29]:
c_list[1]

19710.0

In [39]:
i=4
json_open= open('TDNet_full_data/API_index/{}.json'.format(int(c_list[i])), 'r')
js = json.load(json_open)
js

{'statusCode': '200',
 'message': None,
 'count': '61',
 'publiclyList': [{'code': '18530',
   'name': '森組',
   'disclosedDate': '2022-06-02',
   'disclosedTime': '08:00:00',
   'handlingType': None,
   'disclosureNumber': '20220601565910',
   'modifiedHistory': '1',
   'title': '第８９回定時株主総会招集ご通知に際してのインターネット開示事項',
   'disclosureItems': ['11737'],
   'pdfGeneralFlag': '1',
   'pdfSummaryFlag': '0',
   'xbrlFlag': '0'},
  {'code': '18530',
   'name': '森組',
   'disclosedDate': '2022-06-01',
   'disclosedTime': '08:00:00',
   'handlingType': None,
   'disclosureNumber': '20220531564700',
   'modifiedHistory': '1',
   'title': '第８９回定時株主総会招集ご通知',
   'disclosureItems': ['11737'],
   'pdfGeneralFlag': '1',
   'pdfSummaryFlag': '0',
   'xbrlFlag': '0'},
  {'code': '18530',
   'name': '森組',
   'disclosedDate': '2022-05-31',
   'disclosedTime': '15:00:00',
   'handlingType': None,
   'disclosureNumber': '20220531564692',
   'modifiedHistory': '1',
   'title': '独立役員届出書',
   'disclosureItems': ['117

In [3]:
import os

path = 'new2/NoPre'

folderfile = os.listdir(path)
cnt=0
for i in folderfile:
    if len(i)==5:
        cnt+=1
print(cnt)

1347


In [292]:
column_list

[['売上高',
  'EBITDA',
  '営業利益',
  '経常利益',
  '親会社株主に帰属する四半期純利益',
  '(ご参考前年同期間との比較)',
  '(％表示は、対前年同期間増減率)'],
 ['1株当たり四半期純利益', '潜在株式調整後1株当たり四半期純利益'],
 ['売上高', '営業利益', '経常利益', '当期純利益', '親会社株主に帰属する当期純利益']]

In [ ]:
for j,col in enumerate(column_list):
    for i in range(1,len(col)):
        if col[i] =="売上高":
            new_col=col[i:]
            column_list[j] =col[:i]
            break
column_list.append(new_col)

In [246]:
df=pd.DataFrame([["円銭"],[52.00],["1株当たり当期純利益"]],index=["None","通期","column"])

In [288]:
print(df1)

                    0     1    2      3    4      5    6      7    8      9
None              百万円     ％  百万円      ％  百万円      ％  百万円      ％  百万円      ％
2021年12月期第2四半期  13678  23.6  698  117.4  624  154.7  489  458.9  346  922.3
2020年12月期第2四半期  11069  None  321   None  245   None   87   None   33   None


In [289]:
df1.loc["None"]

0    百万円
1      ％
2    百万円
3      ％
4    百万円
5      ％
6    百万円
7      ％
8    百万円
9      ％
Name: None, dtype: object

In [312]:
df1=pd.DataFrame([n,n1,n2,column],index=["None","2021年12月期第2四半期","2020年12月期第2四半期","column"])

In [313]:
n=["百万円","％","百万円","％","百万円","%","百万円","%","百万円","％","円銭","円銭"]

In [314]:
n1=[13678,23.6,698,117.4,624,154.7,489,458.9,346,922.3,8.34,8.27]
n2=[11069,"None",321,"None",245,"None",87,"None",33,"None",0.87,0.86]

In [315]:
column=['売上高',
        '売上高',
  'EBITDA',
         'EBITDA',
  '営業利益',
        '営業利益',
  '経常利益',
        '経常利益',
  '親会社株主に帰属する四半期純利益','親会社株主に帰属する四半期純利益',"1株当たり当期純利益","潜在株式調整後１株当たり四半期純利益"]

In [331]:
col=[df1.loc["index"][i] for i in range(0,12)]

In [341]:
n=["百万円","％","百万円","％","百万円","%","百万円","%","百万円","％","百万円","％","円銭"]
column=['売上高',
        '売上高',
  'EBITDA',
         'EBITDA',
  '営業利益',
        '営業利益',
  '経常利益',
        '経常利益',
  '親会社株主に帰属する四半期純利益','親会社株主に帰属する四半期純利益']

In [350]:
n1=[22773,9.0,1007,-7.8,873,-25.9,515,-22.7,22773,]
n2=[4912,23.0,13779,36.0,-1650,"None",-2150,"None",-2250,"None",-2300,"None",-42.995]

In [351]:
df=pd.DataFrame([n,n1,n2,col],index=["None","通期","第2四半期","column"])

In [352]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
None,百万円,％,百万円,％,百万円,%,百万円,%,百万円,％,百万円,％,円銭
通期,21104,35.0,16279.5,45.0,None,None,None,None,None,None,None,None,None
第2四半期,4912,23.0,13779,36.0,-1650,None,-2150,None,-2250,None,-2300,None,-42.995
column,売上高,売上高,SaaSARR※1,SaaSARR※1,EBITDA※2,EBITDA※2,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する四半期純利益,親会社株主に帰属する四半期純利益,1株当たり四半期純利益


In [338]:
col=['売上高',
 '売上高',
 'SaaSARR※1',
 'SaaSARR※1',
 'EBITDA※2',
 'EBITDA※2',
 '営業利益',
 '営業利益',
 '経常利益',
 '経常利益',
 '親会社株主に帰属する四半期純利益',
'親会社株主に帰属する四半期純利益',
 '1株当たり四半期純利益']

In [367]:
col=['売上高','売上高', '営業利益', '営業利益','経常利益','経常利益', '親会社株主に帰属する四半期純利益','親会社株主に帰属する四半期純利益','1株当たり四半期純利益', '潜在株式調整後1株当たり四半期純利益', 'EBITDA※1','EBITDA※1', 'EBIT※2', 'EBIT※2'],

In [366]:
df1

,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,8,9,10,11
None,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％
30年12月期第1四半期,22773,9.0,1007,-7.8,873,-25.9,515,-22.7,22773,18.9,1007,-2.2,873,-21.8,515,-19.1,1265,-17.4,893,-20.4
29年12月期第1四半期,20886,12.6,1093,23.0,1178,44.2,666,269.3,19158,None,1030,None,1117,None,637,None,1531,None,1121,None


In [370]:
colu=pd.DataFrame(col,index=["column"])

In [375]:
df5=pd.concat([df1,colu])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [377]:
df1

,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,8,9,10,11
None,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％,百万円,％
30年12月期第1四半期,22773,9.0,1007,-7.8,873,-25.9,515,-22.7,22773,18.9,1007,-2.2,873,-21.8,515,-19.1,1265,-17.4,893,-20.4
29年12月期第1四半期,20886,12.6,1093,23.0,1178,44.2,666,269.3,19158,None,1030,None,1117,None,637,None,1531,None,1121,None


In [374]:
df

,0,1,2,3,4,5,6,7,8
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
第2四半期(累計),None,None,None,None,None,None,None,None,None
通期,91000,10.2,2100,-5.6,1500,-21.9,600,-40.7,26.53
